In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas.core.frame import DataFrame
from scipy import stats

In [2]:
data = pd.read_csv('data\\data.csv', sep=';')
data.head()

,Z1-4,Z5A,Z5B,Z6I,Z6IIA,Z6IIB,Z7A,Z7B,Z8A,Z8B,Z9A,Z9B,Z10A,Z10B,Z11,Z12,Z13,"Z14,15,16_x","Z14,15,16_y",Unnamed: 19
0,122.3,152.0,158.8,B,133.7,131.6,168.3,172.2,54.9,76.3,85.8,154.7,24.7,27.7,103.3,100.7,118.5,120.6,84.4,NaN
1,117.9,161.4,159.2,B,170.7,157.0,173.2,154.7,50.9,60.9,134.8,137.2,18.4,20.4,93.9,91.5,122.4,124.2,76.9,NaN
2,118.0,140.1,151.5,B,169.2,155.3,186.1,167.4,28.2,66.2,174.0,146.7,29.5,30.4,96.3,96.5,120.4,126.2,82.3,NaN
3,124.0,147.0,160.4,B,163.1,148.2,160.7,176.6,54.2,89.9,177.9,142.9,30.2,25.8,92.2,94.3,117.9,127.6,80.0,NaN
4,126.5,150.6,152.2,B,144.1,143.4,177.9,160.7,42.7,68.9,163.4,136.3,20.7,26.0,97.4,104.7,120.3,128.9,76.7,NaN


# Интервальные оценки

## Интервальная оценка для неизвестного математического ожидания

В эксперименте наблюдается выборка объёма n из копий случайной величины X. Предполагая, что распределение X нормально, необходимо построить доверительный интервал (верхнюю или нижнюю границу) для неизвестного математического ожидания µ = E X.

In [60]:
def conf_interval_mean(x_data: np.array,
                       Q: float=0.99,
                       type_interval='lower'):
    """
    Интервальная оценка для мат. ожидание
    Args:
        x_data: данные
        Q: надежность интервала
        type_interval (lower, upper, double-side): lower - нижняя,
                                                    upper - верхняя,
                                                    double-side - двухстороняя
    """
    if Q < 0 or Q > 1:
        raise Exception('Q should be [0, 1]!')
    alp = 1 - Q
    n = x_data.shape[0]
    
    if type_interval == 'lower' or type_interval == 'upper':
        t_alp = stats.t.ppf(1 - alp, df=n - 1)
    elif type_interval == 'double-side':
        t_alp = stats.t.ppf(1 - alp / 2, df=n - 1)
    else:
        raise Exception('Type interval should be like lower, upper, double-side!')
    
    m = x_data.std() / math.sqrt(n - 1)
    x_mean = x_data.mean()
    
    print(f"Надежность {Q * 100}%")
    
    if type_interval == 'lower':
        print(f"Нижняя граница среднего: {x_mean - m * t_alp}")
    elif type_interval == 'upper':
        print(f"Верхняя граница среднего: {x_mean + m * t_alp}")
    else:
        print(f"Двухстороняя граница среднего: ({x_mean - m * t_alp}, {x_mean + m * t_alp})")

In [61]:
X_data = data['Z11'].to_numpy()
X_data = X_data[~np.isnan(X_data)]

conf_interval_mean(X_data, type_interval='double-side')

Надежность 99.0%
Двухстороняя граница среднего: (96.89914358217128, 101.88418975116208)


# Интервальная оценка для неизвестной дисперсии нормального распределения

В эксперименте наблюдается выборка объёма n из копий случайной величины X. Предполагая, что распределение X нормально, необходимо построить доверительный интервал (верхнюю или нижнюю границу) для неизвестной дисперсии σ 2 = DX и стандартного отклонения σ.

In [58]:
def conf_interval_var(x_data: np.array,
                       Q: float=0.99,
                       type_interval='lower'):
    """
    Интервальная оценка для дисперсии
    Args:
        x_data: данные
        Q: надежность интервала
        type_interval (lower, upper, double-side): lower - нижняя,
                                                    upper - верхняя,
                                                    double-side - двухстороняя
    """
    if Q < 0 or Q > 1:
        raise Exception('Q should be [0, 1]!')
    alp = 1 - Q
    n = x_data.shape[0]
    
    if type_interval == 'lower':
        chi2_alp = stats.chi2.ppf(1 - alp, df=n - 1)
    elif type_interval == 'upper':
        chi2_alp = stats.chi2.ppf(alp, df=n - 1)
    elif type_interval == 'double-side':
        chi2_alp_lower = stats.chi2.ppf(1 - alp / 2, df=n - 1)
        chi2_alp_upper = stats.chi2.ppf(alp / 2, df=n - 1)
    else:
        raise Exception('Type interval should be like lower, upper, double-side!')
    
    x_var = x_data.var()
    
    print(f"Надежность {Q * 100}%")
    
    if type_interval == 'lower':
        print(f"Нижняя граница дисперсии: {n * x_var / chi2_alp}")
    elif type_interval == 'upper':
        print(f"Верхняя граница дисперсии: {n * x_var / chi2_alp}")
    else:
        print(f"Двухстороняя граница дисперсии: ({n * x_var / chi2_alp_lower},\
{n * x_var / chi2_alp_upper})")

In [59]:
X_data = data['Z12'].to_numpy()
X_data = X_data[~np.isnan(X_data)]

conf_interval_var(X_data, Q=0.9, type_interval='upper')

Надежность 90.0%
Верхняя граница дисперсии: 37.91692015092335


# Интервальная оценка для вероятности успеха

В эксперименте подсчитывается число успешных реализаций некоторого события (например, число доброкачественных изделий). Требуется построить доверительный интервал (верхнюю или нижнюю границу) для истинной вероятности p этого события.

In [56]:
def conf_interval_prob(x_data: np.array,
                       Q: float=0.99,
                       type_interval='lower'):
    """
    Интервальная оценка для вероятности успеха
    Args:
        x_data: данные
        Q: надежность интервала
        type_interval (lower, upper, double-side): lower - нижняя,
                                                    upper - верхняя,
                                                    double-side - двухстороняя
    """
    if Q < 0 or Q > 1:
        raise Exception('Q should be [0, 1]!')
    alp = 1 - Q
    n = x_data.shape[0]
    
    if type_interval == 'lower' or type_interval == 'upper':
        t_alp = stats.norm.ppf(1 - alp)
    elif type_interval == 'double-side':
        t_alp = stats.norm.ppf(1 - alp / 2)
    else:
        raise Exception('Type interval should be like lower, upper, double-side!')
    
    p_data = sum(x_data) / n
    m = math.sqrt(p_data * (1 - p_data) / n)
    
    print(f"Надежность {Q * 100}%")
    
    if type_interval == 'lower':
        print(f"Нижняя граница вероятности успеха: {p_data - m * t_alp}")
    elif type_interval == 'upper':
        print(f"Верхняя граница вероятности успеха: {p_data + m * t_alp}")
    else:
        print(f"Двухстороняя граница вероятности успеха: ({p_data - m * t_alp},\
{p_data + m * t_alp})")

In [57]:
X_data = data['Z13'].to_numpy()
X_data = X_data[~np.isnan(X_data)]

conf_interval_prob(X_data > 116.5, Q=0.975, type_interval='double-side')

Надежность 97.5%
Двухстороняя граница вероятности успеха: (0.902589384850798,1.0043873593352486)
